# Chapter 8: Metaclasses and Attributes
Follow the rule of least surprise

## Item 58: Use Plain Attributes Instead of Setter and Getter Methods
Java practitioners, take note!

In [ ]:
from time import sleep


class Jimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

    def get_fios_installed(self):
        return self.fios_installed

    def set_fios_installed(self, setto: bool):
        self.fios_installed = setto

    # etc etc etc

In [ ]:
jimbo = Jimmy(False)
jimbo.set_fios_installed(True)
jimbo.get_fios_installed()

In [ ]:
# But for simple cases like this, getter/setters aren't needed:
class PythonicJimmy:
    def __init__(self, fios_installed: bool):
        self.fios_installed = fios_installed

pimbo = PythonicJimmy(False)
pimbo.fios_installed = True
pimbo.fios_installed

In [ ]:
# If we do need a more complicated interface for interacting with an attribute, use @property
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [ ]:
jimbo2 = Jimmy(0)
for x in range(0, 9):
    jimbo2.spongebob_references = x

In [ ]:
# This also lets you do type checking at runtime
class Jimmy:
    def __init__(self, spongebob_refs: int):
        self.sb_refs = spongebob_refs
        self.time_til_patrick = 10

    @property
    def spongebob_references(self):
        return self.sb_refs

    @spongebob_references.setter
    def spongebob_references(self, references: int):
        if not isinstance(references, int):
            raise ValueError("NO")
        if (self.time_til_patrick > 0):
            self.sb_refs = references
            self.time_til_patrick -= 1
        else:
            raise ValueError("Jimmy is now doing a Patrick impression")

In [ ]:
jimbo = Jimmy(0)
jimbo.spongebob_references = 0.5

# NOTE: you could also enforce immutability this way: raise an exception after a `hasattr` check

### Remember
- Define new class interfaces using simple public attributes and avoid defining setter and getter methods.
- Use @property to define special behavior when attributes are accessed on your objects.
- Follow the rule of least surprise and avoid odd side effects in your @property methods.
- Ensure that @property methods are fast; for slow or complex work—especially involving I/O or causing side effects—use normal methods instead.

## Item 59: Consider `@property` Instead of Refactoring Attributes

In [ ]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f"Bucket(quota={self.quota})"

In [ ]:
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False  # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False  # Bucket was filled, but not enough
    bucket.quota -= amount
    return True       # Bucket had enough, quota consumed

In [ ]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

In [ ]:
if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print(bucket)

In [ ]:
if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print(bucket)


# Problem here, is we never really know if we're blocked because we ran out of quota, or because we never had quota (during this time period)

In [ ]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0 # we add a max_quota to track quota issued during the period
        self.quota_consumed = 0 # and this one is, well, how much we used dawg (dawg)

    def __repr__(self):
        return (
            f"NewBucket(max_quota={self.max_quota}, "
            f"quota_consumed={self.quota_consumed})"
        )

    # And NOW we use property so this new bucket has a compatible interface!
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled during the period
            self.max_quota = amount + self.quota_consumed
        else:
            # Quota being consumed during the period
            self.quota_consumed = delta

In [ ]:
bucket = NewBucket(60)
print("Initial", bucket)
fill(bucket, 100)
print("Filled", bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print("Now", bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print("Still", bucket)


The magic of `@property` is it lets you make incremental progress towards a better data model.

> @property is a tool to help you address problems you’ll come across in real-world code. Don’t overuse it. When you find yourself repeatedly extending @property methods, it’s probably time to refactor your class instead of further paving over your code’s poor design.

# Item 60: Use Descriptors for Reusable `@property` Methods

Its kinda like Mixins, but for attributes.

In [ ]:
# Note 2 eric: show the preamble in the book

class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._value = value

class Exam:
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [ ]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print("Writing", first_exam.writing_grade)
print("Science", first_exam.science_grade)

In [ ]:
# All good right? NOPE!!!!
second_exam = Exam()
second_exam.writing_grade = 75
print(f"Second {second_exam.writing_grade} is right")
print(f"First  {first_exam.writing_grade} is wrong; "
      f"should be 82")


In [ ]:
# Why this happen???
second_exam.writing_grade is first_exam.writing_grade

Remember long ago:
```python
def foo(bar=[]):
    ...
```

Like this older example, Python only evaluates that `Grade()` once!

In [ ]:
class DictGrade:
    """
    A Grade class that keeps track of the individual values in each Exam that uses it.
    """
    def __init__(self):
        self._values = {}  # hint

    def __get__(self, instance, instance_type):
        """Return the grade value for the instance"""
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        """Set the grade value for the instance"""
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._values[instance] = value

Put on your Garbage Collector Engineer hat: what is wrong with the above implementation?

In [ ]:
# Luckily Python actually has a way of making a Descriptor that can be unique to the instance that uses it:
# BEHOLD: __set_name__
class NamedGrade:
    def __set_name__(self, owner, name):
        self.internal_name = "_" + name  # creates a unique attr just for the "owning" instance

    def __get__(self, instance, instance_type):
        """Return the grade value for the instance"""
        if instance is None:
            return self
        return getattr(instance, self.internal_name)  # Note how we use getattr here,

    def __set__(self, instance, value):
        """Set the grade value for the instance"""
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        setattr(instance, self.internal_name, value)  # Note setattr

In [ ]:
# Watch the magic

class NamedExam:
    math_grade = NamedGrade()
    writing_grade = NamedGrade()
    science_grade = NamedGrade()

vars(NamedExam.science_grade)

In [ ]:
first_exam = NamedExam()
first_exam.math_grade = 78
first_exam.writing_grade = 89
first_exam.science_grade = 94
first_exam.__dict__

In [ ]:
# And just double checking our instance tracking works..
second_exam = NamedExam()
second_exam.writing_grade = 99
first_exam.writing_grade is second_exam.writing_grade

Okay, so how is it working? Take a look back to `setattr` and `getattr`. The descriptor _itself_ is not storing the data, rather, the data is now being stored on `Exam`, but with the `internal_name` that the Descriptor knows it by (for Exam instances at least).

The `writing_grade` Descriptor now knows, when an Exam class is calling `set` on it, it needs to reroute that `get` to an internal field on `Exam` called `_writing_grade`. Likewise with sets!

Now the `NamedGrade` no longer keeps a reference to any `Exam` objects, allowing full garbage collection!

### Remember
-  Reuse the behavior and validation of @property methods by defining your own descriptor classes.
- Use __set_name__ along with setattr and getattr to store the data needed by descriptors in object instance dictionaries in order to avoid memory leaks.
- Don’t get bogged down trying to understand exactly how __getattribute__ uses the descriptor protocol for getting and setting attributes.

# Item 61: Use `__getattr__`, `__getattribute__`, and `__setattr__` for Lazy Attributes
We saw getattr and setattr already, lets take a gander at these more, and why they're sweet

In [ ]:
# When a class HAS a __getattr__ defined, it is called whenever an attribute can't be found
# Imagine this is the Python representation of a database record.

class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        """If they're trying to find an attr that doesn't exist, we just make it!"""
        value = f"Value for {name}"
        setattr(self, name, value)
        return value

In [ ]:
data = LazyRecord()
print("Before:", data.__dict__)
print("foo:   ", data.foo)
print("After: ", data.__dict__)

In [ ]:
# An example of attaching some self-logging, to show where the getattr actually happens

class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(
            f"* Called __getattr__({name!r}), "
            f"populating instance dictionary"
        )
        result = super().__getattr__(name)  # POP QUIZ: Why am I using super here?
        print(f"* Returning {result!r}")
        return result

data = LoggingLazyRecord()
print("exists:     ", data.exists)
print("First foo:  ", data.foo)
print("Second foo: ", data.foo)

Note that the log only happened once, right before the `First foo`. By the time we try to print foo again, it exists on the class, so `__getattr__` is not called again. **Lazy Instantiation**!

In [ ]:
# What about __getattribute__?
# Imagine we want to support db Transactions: anytime a user accesses an attr, we want to check if its valid, AND if the transaction is still open!
# __getattr__ won't help us here, cause it only gets called that first time an attr is being made

class ValidatingRecord:
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name): # This gets called anytime an attr is accessed
        print(f"* Called __getattribute__({name!r})")
        try:
            value = super().__getattribute__(name)
            print(f"* Found {name!r}, returning {value!r}")
            return value
        except AttributeError:  # if the attr doesn't exist, lets go ahead and add it!
            value = f"Value for {name}"
            print(f"* Setting {name!r} to {value!r}")
            setattr(self, name, value)
            return value

data = ValidatingRecord()
print("exists:     ", data.exists)
print("First foo:  ", data.foo)
print("Second foo: ", data.foo)

In [ ]:
# Speaking of AttributeError, this is exaclty what this exception is for

class MissingPropertyRecord:
    def __getattr__(self, name):
        if name == "bad_name":
            raise AttributeError(f"{name} is missing")
        ...

data = MissingPropertyRecord()
data.bad_name

Use `AttributeError` to activate Python's standard handling of a missing attribute.

In [ ]:
# Side note, `hasattr` is useful too
data = LoggingLazyRecord()  # Implements __getattr__
print("Before:         ", data.__dict__)
print("Has first foo:  ", hasattr(data, "foo"))
print("After:          ", data.__dict__)
print("Has second foo: ", hasattr(data, "foo"))

Notice something strange? Why did the getattr logger fire? 👀

`hasattr` invoked `__getattr__` since foo didn't exist.

In [ ]:
# A little more about setattr: unlike getattr and getattribute, theres no need for two set methods, setattr handles both cases

class SavingRecord:
    def __setattr__(self, name, value):
        # Imagine those ... are fancy code that magically updates the database in the background as attrs are set on this class
        ...
        super().__setattr__(name, value)

# Lets take a look at call order
class LoggingSavingRecord(SavingRecord):
    def __setattr__(self, name, value):
        print(f"* Called __setattr__({name!r}, {value!r})")
        super().__setattr__(name, value)

data = LoggingSavingRecord()
print("Before: ", data.__dict__)
data.foo = 5
print("After:  ", data.__dict__)
data.foo = 7
print("Finally:", data.__dict__)

In [ ]:
# SURPRISE POP QUIZ: find the error
class BrokenDictionaryRecord:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print(f"* Called __getattribute__({name!r})")
        return self._data[name]

data = BrokenDictionaryRecord({"foo": 3})
data.foo

# Remember
- Use __getattr__ and __setattr__ to lazily load and save attributes for an object.
- Understand that __getattr__ only gets called when accessing a missing attribute, whereas __getattribute__ gets called every time any attribute is accessed.
- Avoid infinite recursion in __getattribute__ and __setattr__ method implementations by calling super().__getattribute__ and super().__getattr__ to access object attributes.

# Item 62: Validate Subclasses with `__init_subclass__`
Metaclasses give a nice way to validate subclasses, enforcing structure and relationships.

In [ ]:
# HOL UP
# WTF even _is_ a metaclass tho

class Meta(type):  # note it inherits from type
    zzz = 1
    def __new__(meta, name, bases, class_dict):
        print(f"* Running {meta}.__new__ for {name}")
        print("Bases:", bases)
        print(class_dict)
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

class MySubclass(MyClass):
    other = 567

    def bar(self):
        pass


Notice something interesting about that last cell?

Yeah: The metaclass ran after the construction of `MyClass`.

`bases` is the info about the parent classes btw. That's why we see `(<class '__main__.MyClass'>,)` under `MySubClass`, but nothing under (parentless) `MyClass` 😀

`meta` is "self"

`name` is the name of the class (not the meta class)

`class_dict` come on you guys should know what this is by now 🦍

#### But there's something else interesting...

In [ ]:
foo = MySubclass()
dir(foo)

Classes inherit from `object`, metaclasses inherit from `type`. Note how `Meta` has a `zzz` field, but none of the classes that utilize it have that field.

Point I'm trying to make: Metaclasses are not strictly part of the class hierarchy, think of them as a controling concern, outside of the classes.

### Okay but who cares?

[Pydantic cares](https://github.com/pydantic/pydantic/blob/d992117243b3d44047ef37f9d7d75243bb98dbca/pydantic/_internal/_model_construction.py#L80)

In [ ]:
# The above is a pretty advanced example. Lets scale back yeah?

class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Polygon class
        if bases:
            if class_dict["sides"] < 3:
                raise ValueError("Polygons need 3+ sides")
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    sides = None  # Must be specified by subclasses

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Triangle(Polygon):
    sides = 3

class Rectangle(Polygon):
    sides = 4

class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260

In [ ]:
# Let's be wrong and bad

print("1")

class Line(Polygon):
    print("2")
    sides = 2
    print("3")

print("4")

### Pretty cool stuff, but seems like a lot of work yeah?

[Pydantic agrees](https://github.com/pydantic/pydantic/blob/2c38bf402c2a7ebadfcf2d1ccf8718513f9dc969/pydantic/root_model.py#L55)

We now have `__init_subclass__` special class method, which can get us the same behavior, and _avoid metaclasses entirely_.

In [ ]:
class BetterPolygon:
    sides = None  # Subclasses must specify this

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError("Polygons need 3+ sides.")

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
    sides = 6

assert Hexagon.interior_angles() == 720

### The pros
- This is a lot easier to read than `ValidatePolygon`. I'd argue the uninitaited pythonista would probably gleam whats happening here without too much thinking, whereas the metaclass could tkae a moment to understand.
- We don't have to mess around with `class_dict` since we're already right here in the `cls`

In [ ]:
print("1")
class Point(BetterPolygon):
    sides = 1
print("2")

### The pros, pt2
Any given class can only have **one** metaclass. I mention this because programmers new to metaclasses will instinctually thing they are superior to `__init_subclass__` via composability; create a variety of shareable validations via metaclass and compose them together!
  - Alas, this is but a dream.

In [ ]:
# Lets show what I mean, here's a second class with its own meta.
# We'll make a Red Pentagon!

class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Filled class
        if bases:
            if class_dict["color"] not in ("red", "green"):
                raise ValueError("Fill color must be supported")
        return type.__new__(meta, name, bases, class_dict)

class Filled(metaclass=ValidateFilled):
    color = None  # Must be specified by subclasses

class RedPentagon(Filled, Polygon):
    color = "blue"
    sides = 5

😔 Alas..

Now.. it _is_ possible to do something like this via a complex class hierarchy, since Metaclasses can inherit from eachother. But that _ain't_ composable.

Imagine trying to use a mixin with this; pain and suffering!

In [ ]:
# Let's redo it with __init_subclass__

class BetterFilled:
    color = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ("red", "green", "blue"):
            raise ValueError("Fills need a valid color")

class RedTriangle(BetterFilled, BetterPolygon):
    color = "red"
    sides = 3

red_boi = RedTriangle()
assert isinstance(red_boi, BetterFilled)
assert isinstance(red_boi, BetterPolygon)

In [ ]:
class BlueLine(BetterFilled, BetterPolygon):
    color = "blue"
    sides = 2

In [ ]:
class ChartreuseSquare(BetterFilled, BetterPolygon):
    color = "chartreuse"
    sides = 4

## Things to Remember

- The `__new__` method of metaclasses is run after the class statement’s entire body has been processed.
- Metaclasses can be used to inspect or modify a class after it’s defined but before it’s created, but they’re often more heavyweight than you need.
- Use `__init_subclass__` to ensure that subclasses are well formed at the time they are defined, before objects of their type are constructed.
- Be sure to call `super().__init_subclass__` from within your class’s `__init_subclass__` definition to enable composable validation in multiple layers of classes and multiple inheritance.

# Item 63: Register Class Existence with `__init_subclass__`
The `__init_subclass__` good times aint over! This method is pretty useful for registering the existence of types. This is handy in scenarios where you need to do a reverse lookup.

Bear with me...

In [ ]:
# Our own little "serializable"
# (though its less cool than that JSON mixin we made, that thing was sweet).

import json

class Serializable:
    def __init__(self, *args):
        # note here, the class records the input arguments
        self.args = args

    def serialize(self):
        return json.dumps({"args": self.args})

# Lets see _why_ we recorded those args
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point2D({self.x}, {self.y})"

point = Point2D(5, 3)
print("Object:    ", point)
print("Serialized:", point.serialize())

Our `Serializable` captures the "args" so it can effectively serialize it!

In [ ]:
# We gotta deserialize sometimes too I guess

class Deserializable(Serializable): # Note the parent
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params["args"])

In [ ]:
class BetterPoint2D(Deserializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point2D({self.x}, {self.y})"

before = BetterPoint2D(5, 3)
print("Before:    ", before)
data = before.serialize()
print("Serialized:", data)
after = BetterPoint2D.deserialize(data)
print("After:     ", after)


The problem here, is you gotta inherit from `Des` anytime you want this functionality. Not terribly generic.. and idk about you, but typically when dealing with JSON, I'm SerDes'ing a bunch of different classes.

Let's attempt to make something _more_ generic, supporting a generic `deserialize` function.

In [ ]:
class BetterSerializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps(
            {
                # include the serialized object’s class name in the JSON data
                "class": self.__class__.__name__,
                "args": self.args,
            }
        )

    def __repr__(self):
        name = self.__class__.__name__
        args_str = ", ".join(str(x) for x in self.args)
        return f"{name}({args_str})"

In [ ]:
REGISTRY = {}  # in practice, possibly a global

def register_class(target_class):
    REGISTRY[target_class.__name__] = target_class

def deserialize(data):
    params = json.loads(data)
    name = params["class"]
    target_class = REGISTRY[name]
    return target_class(*params["args"])

In [ ]:
class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

# I have to do this every time if I want `deserialize` to support it
register_class(EvenBetterPoint2D)
# Surely you see where this is going...

In [ ]:
before = EvenBetterPoint2D(5, 3)
print("Before:    ", before)
data = before.serialize()
print("Serialized:", data)
after = deserialize(data)
print("After:     ", after)

It works.

But it sucks too: we, and our callers, are probably going to forget to call `register_class(...)`

- (I mean... we could probably solve this with a metaclass... put `register_class` in `__new__` 👀)
- (( But we already saw that `__init_subclass__` is better more useful for this kind of thing ))
- ((( Look at all these parens, what is this, a lisp dialect?! )))
- (((( Okay, lets look at a better way ))))

In [ ]:
class BetterRegisteredSerializable(BetterSerializable):
    def __init_subclass__(cls):
        super().__init_subclass__()
        register_class(cls)

class Vector1D(BetterRegisteredSerializable):
    def __init__(self, magnitude):
        super().__init__(magnitude)
        self.magnitude = magnitude

before = Vector1D(6)
print("Before:    ", before)
data = before.serialize()
print("Serialized:", data)
print("After:     ", deserialize(data))

I got lazy and didn't want to look, but its likely you'll find this pattern in popular ORM libraries like `Beanie` and `SQLAlchemy`.

## Things to Remember

- Class registration is a helpful pattern for building modular Python programs.
- Metaclasses let you run registration code automatically each time your base class is subclassed in a program.
- Using metaclasses for class registration helps you avoid errors by ensuring that you never miss a registration call.
- Prefer `__init_subclass__` over standard metaclass machinery because it’s clearer and easier for beginners to understand.

# Item 64: Annotate Class Atributes with `__set_name__`
Not gonna lie, I found this section largely redundant to Item 60.

This chapter shows how to accomplish a descriptor via a metaclass, and _then_ shows you to not do that and instead do it with `__set_name__`, the way Item 60 shows.

I'm just gonna skip this one!

# Item 65: Consider Class Body Definition Order to Establish Relationships Between Attributes

In [ ]:
# Lots of times python represents external data.
# Lets look at some external data!

import csv

with open("packages.csv") as f:
    for row in csv.reader(f):
        print(row)

# Side note: isn't python's csv package great? It just works

In [ ]:
# That data is useless to us right now, lets do something with it!

class Delivery:
    def __init__(self, destination, method, weight):
        self.destination = destination
        self.method = method
        self.weight = weight

    @classmethod
    def from_row(cls, row):
        """Sweet sweet static constructor"""
        return cls(row[0], row[1], row[2])

row1 = ["Sydney", "truck", "25"]
obj1 = Delivery.from_row(row1)
print(obj1.__dict__)

Good stuff. But what if we have a _lot_ of different kinds of CSVs? Do we gotta write `from_row` and the `init` for every one?!

😡 don't yall know I'm lazy

In [ ]:
# We can save a bit of effort via `fields` :D

class RowMapper:
    """A parent class for CSV eaters!"""
    fields = ()  # Must be in CSV column order

    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            if key not in type(self).fields:
                raise TypeError(f"Invalid field: {key}")
            setattr(self, key, value)

    @classmethod
    def from_row(cls, row):
        if len(row) != len(cls.fields):
            raise ValueError("Wrong number of fields")
        kwargs = dict(pair for pair in zip(cls.fields, row))
        return cls(**kwargs)

In [ ]:
# Behold, Delivery, but now a mapper
class DeliveryMapper(RowMapper):
    fields = ("destination", "method", "weight")

obj2 = DeliveryMapper.from_row(row1)
assert obj2.destination == "Sydney"
assert obj2.method == "truck"
assert obj2.weight == "25"

In [ ]:
# We can make more!!!
class MovingMapper(RowMapper):
    fields = ("source", "destination", "square_feet")

It's got a weakness though. Attributes defined via string? MEH

Also, using `fields` feels kinda redundant. It's a list of attributes inside a list of attributes. I don't like it.

I'd rather actually have the field names, in a python-y way. LETS DO THIS:

In [ ]:
class BetterRowMapper(RowMapper):
    def __init_subclass__(cls): # this bad boi again
        fields = []
        for key, value in cls.__dict__.items():  # dict insertion order is consistent in new python
            if value is Ellipsis: # 👀
                fields.append(key)
        cls.fields = tuple(fields)

In [ ]:
class BetterDeliveryMapper(BetterRowMapper):
    destination = ...
    method = ...
    weight = ...

obj3 = BetterDeliveryMapper.from_row(row1)
assert obj3.destination == "Sydney"
assert obj3.method == "truck"
assert obj3.weight == "25"

Much nicer! If we see `...` we know that the subclass is meant to instantiate a field with that name.

The dict-ordering stuff (from item 25) also lets us move fields around (in case the CSV's change their ordering):

In [ ]:
class ReorderedDeliveryMapper(BetterRowMapper):
    method = ...
    weight = ...
    destination = ...  # Moved

row4 = ["road train", "90", "Perth"]  # Different order
obj4 = ReorderedDeliveryMapper.from_row(row4)
print(obj4.__dict__)

This works well, but using `Ellipsis` can be a bit clunky when you want more advanced stuff like validation, data conversion, etc... If only we had a way to have fields with advanced behaviors on a class...

🤔

In [ ]:
# Descriptors again?!

class Field:
    def __init__(self):
        self.internal_name = None

    def __set_name__(self, owner, column_name):
        self.internal_name = "_" + column_name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, "")

    def __set__(self, instance, value):
        adjusted_value = self.convert(value)
        setattr(instance, self.internal_name, adjusted_value)

    def convert(self, value):
        raise NotImplementedError

# Basically, you could use this instead of Ellipsis

class StringField(Field):
    def convert(self, value):
        if not isinstance(value, str):
            raise ValueError
        return value

class FloatField(Field):
    def convert(self, value):
        return float(value)

class DescriptorRowMapper(RowMapper):
    def __init_subclass__(cls):
        fields = []
        for key, value in cls.__dict__.items():
            if isinstance(value, Field):  # Changed
                fields.append(key)
        cls.fields = tuple(fields)

In [ ]:
class ConvertingDeliveryMapper(DescriptorRowMapper):
    destination = StringField()
    method = StringField()
    weight = FloatField()

obj5 = ConvertingDeliveryMapper.from_row(row1)
assert obj5.destination == "Sydney"
assert obj5.method == "truck"
assert obj5.weight == 25.0  # Number, not string

Side quest, lets write our own Temporal, using this stuff.

In [ ]:
def activity(func):
    """Super basic decorator, if we were serious we'd use functools.wrap"""
    func._is_activity = True
    return func

class Workflow:
    def __init_subclass__(cls):
        activities = []
        for key, value in cls.__dict__.items(): # like fields, order is enforced
            if callable(value) and hasattr(value, "_is_activity"):
                activities.append(key)
        cls.activities = tuple(activities)

    def run(self):
        for activity_name in type(self).activities:
            func = getattr(self, activity_name)
            func()

In [ ]:
from time import sleep
class ResizerSupportWorkflow(Workflow):
    @activity
    def page_jimmy(self):
        print("paging Jimmy")
        sleep(1)

    def crappy_helper_function(self):
        raise RuntimeError("Whoops this function sucks!")

    @activity
    def figure_out_if_pebkac(self):
        print("determining if its the customers fault")
        sleep(3)

    @activity
    def apologize_to_jimmy(self):
        print("lol sorry Jimmy, it was the customers fault.")

In [ ]:
workflow = ResizerSupportWorkflow()
workflow.run()

## Things to Remember

- You can examine the attributes and methods defined in a class body at runtime by inspecting the corresponding class object’s `__dict__` instance dictionary.
- The definition order of class bodies is preserved in a class object’s `__dict__`, enabling code to consider the relative positions of a class’s attributes and methods. This is especially useful for use cases like mapping object fields to CSV column indexes.
- Descriptors and method decorators can be used to further enhance the power of using the definition order of class bodies to control program behavior.

# Item 66: Prefer Class Decorators over Metaclasses for Composable Class Extensions
Continuing the theme of "plz bro, plz don't use metaclasses bro"

In [ ]:
# Let's make our own Datadog tracer

from functools import wraps

def trace_func(func):
    if hasattr(func, "tracing"):  # Only decorate once
        return func

    @wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = repr(args)
        kwargs_repr = repr(kwargs)
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(
                f"{func.__name__}"
                f"({args_repr}, {kwargs_repr}) -> "
                f"{result!r}"
            )

    wrapper.tracing = True
    return wrapper

In [ ]:
class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        return super().__init__(*args, **kwargs)

    @trace_func
    def __setitem__(self, *args, **kwargs):
        return super().__setitem__(*args, **kwargs)

    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)

In [ ]:
trace_dict = TraceDict([("hi", 1)])
trace_dict["there"] = 2
trace_dict["hi"]
try:
    trace_dict["does not exist"]
except KeyError:
    pass  # Expected

All well and good.. but this will immediately become annoying if we add more stuff to `TraceDict`, because then we need to add `@trace_func` to the new stuff. Ugh!

We could _attempt_ to solve this with metaclasses (crowd booing noises)

In [ ]:
import types

TRACE_TYPES = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType,
    types.WrapperDescriptorType,
)

IGNORE_METHODS = (
    "__repr__",
    "__str__",
)

class TraceMeta(type):
    """A metaclass that automatically decorates all methods of a class"""
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            if key in IGNORE_METHODS:
                continue

            value = getattr(klass, key)
            if not isinstance(value, TRACE_TYPES):
                continue

            wrapped = trace_func(value)
            setattr(klass, key, wrapped)

        return klass

In [ ]:
# Behold
class TraceDict(dict, metaclass=TraceMeta):
    pass

trace_dict = TraceDict([("hi", 1)])
trace_dict["there"] = 2
trace_dict["hi"]
try:
    trace_dict["does not exist"]
except KeyError:
    pass  # Expected

Naively, this works fine. But there is one achilles heel: what if you try to trace a class that has a parent with its own metaclass? Collision.

Instead, a better way is to use class decorators.

Yes thats right, its not just functions that can be decorated.
![](decorators.png)

In [ ]:
def my_class_decorator(klass):
    klass.extra_param = "hello"
    return klass

@my_class_decorator
class MyClass:
    pass

print(MyClass)
print(MyClass.extra_param)

In [ ]:
# Datadog Center, take two

def trace(klass):
    for key in dir(klass):
        if key in IGNORE_METHODS:
            continue

        value = getattr(klass, key)
        if not isinstance(value, TRACE_TYPES):
            continue

        wrapped = trace_func(value)
        setattr(klass, key, wrapped)

    return klass

In [ ]:
# Behold

@trace
class DecoratedTraceDict(dict):
    pass

trace_dict = DecoratedTraceDict([("hi", 1)])
trace_dict["there"] = 2
trace_dict["hi"]
try:
    trace_dict["does not exist"]
except KeyError:
    pass  # Expected

And, class decorators work fine if the decorated class already has a metaclass.

## Things to Remember

- A class decorator is a simple function that receives a class instance as a parameter and returns either a new class or a modified version of the original class.
- Class decorators are useful when you want to modify every method or attribute of a class with minimal boilerplate.
- Metaclasses can’t be composed together easily, although many class decorators can be used to extend the same class without conflicts.